tgb - 4/8/2019 
- This script aims at predicting convective heating/moistening, longwave cooling, shortwave heating and microphysical tendencies based on the 8 column SPCAM simulation, with the goal of testing generalization abilities of different network architectures.  
- Notably, we would like to compare the generalization abilities of the conserving network (C), the unconstrained network (U) and the weakly-constrained network (W) with climate change (trained & validated on 0K and then statistics on the +4K simulation).  
- Another aspect is the "data greediness" of each network. Do we need less data when we train with constraints?
Notebook 009 follows the notebook 005 that predicts:
***
[PHQ, PHCLDLIQ, PHCLDICE, TPHYSTND, QRL, QRS, DTVKE, FSNT, FSNS, FLNT, FLNS, PRECT, PRECTEND, PRECST, PRECSTEN] as a function of:  
[QBP, QCBP, QIBP, TBP, VBP, Qdt_adiabatic, QCdt_adiabatic, QIdt_adiabatic, Tdt_adiabatic, Vdt_adiabatic, PS, SOLIN, SHFLX, LHFLX] 

## 1) Preprocess all the necessary variables

In [1]:
#!ln -s /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/cbrain \
#/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()
TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

/home/t/Tom.Beucler/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM


tgb - 4/8/2019 - We change the pre-processing configuration file "8col_rad_tbeucler_local.yml" & because we now train on the 8 column data, we change the prefix to 8col009_01_train.  
Follows the April 4th, 2019 commit of https://github.com/raspstephan/CBRAIN-CAM/blob/master/quickstart.ipynb

In [2]:
#!python preprocessing.py -c pp_config/8col_rad_tbeucler_local.yml

tgb - 4/10/2019 - Process future climate data (+4K experiment)

In [30]:
!python preprocessing.py -c pp_config/8col_rad_tbeucler_p4K.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
04/11/2019 04:14:50 AM Preprocess training dataset
04/11/2019 04:14:50 AM Start preprocessing file /local/Tom.Beucler/SPCAM_PHYS/8col009_02_train.nc
04/11/2019 04:14:50 AM Reading input files
04/11/2019 04:14:50 AM Crop levels
04/11/2019 04:14:50 AM Create stacked dataarray
These time steps are cut: []
04/11/2019 04:14:51 AM Stack and reshape dataarray
Traceback (most recent call last):
  File "preprocessing.py", line 79, in <module>
    main(args)
  File "preprocessing.py", line 29, in main
    preprocess(args.in_dir, args.in_fns, args.out_dir, args.out_fn, args.vars)
  File "/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/cbrain/preprocessing/convert_dataset.py", line 191, in preprocess
    da = reshape_da(da).reset_index('sample')
  File "/filer/z-sv-p

tgb - 4/9/2019 - Follows method adapted by Stephan in his quickstart to define inputs/outputs/normalization

In [3]:
# 1) Define inputs and outputs
in_vars = ['QBP', 'QCBP', 'QIBP', 'TBP', 'VBP', 
           'Qdt_adiabatic', 'QCdt_adiabatic', 'QIdt_adiabatic', 'Tdt_adiabatic', 'Vdt_adiabatic',
           'PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ', 'PHCLDLIQ', 'PHCLDICE', 'TPHYSTND', 'QRL', 'QRS', 'DTVKE', 
            'FSNT', 'FSNS', 'FLNT', 'FLNS', 'PRECT', 'PRECTEND', 'PRECST', 'PRECSTEN']

# 2) Calculates pressure and pressure increments
# Takes representative value for PS since purpose is normalization
PS = 1e5; P0 = 1e5;
P = P0*hyai+PS*hybi; # Total pressure [Pa]
dP = P[1:]-P[:-1]; # Differential pressure [Pa]
dt = 30*60; # timestep

# 3) Define the scaling dictionary 
scale_dict = {
    'PHQ': L_V*dP/G, 
    'PHCLDLIQ': L_V*dP/G, 
    'PHCLDICE': L_V*dP/G, 
    'TPHYSTND': C_P*dP/G, 
    'QRL': C_P*dP/G, 
    'QRS': C_P*dP/G, 
    'DTVKE': C_P*dP/(G*dt), 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
    'PRECT': RHO_L*L_V, 
    'PRECTEND': 1e-3*RHO_L*L_V, 
    'PRECST': RHO_L*L_V, 
    'PRECSTEN': 1e-3*RHO_L*L_V
}

# 4) Save dictionary as pickle file for loading in teh training script
save_pickle('./nn_config/scale_dicts/009_Wm2_scaling.pkl', scale_dict)

## 2) Create data generator and produce data sample

In [4]:
train_gen = DataGenerator(
    data_fn = TRAINDIR+PREFIX+'train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+PREFIX+'norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [5]:
X, Y = train_gen[0]; X.shape, Y.shape

((1024, 304), (1024, 218))

In [6]:
valid_gen = DataGenerator(
    data_fn = TRAINDIR+PREFIX+'valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+PREFIX+'norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=False
)

In [7]:
X, Y = valid_gen[0]; X.shape, Y.shape

((1024, 304), (1024, 218))

# 3) Build neural networks

## 3.1) Conserving Network (C)

In [8]:
inpC = Input(shape=(304,))
densout = Dense(512, activation='linear')(inpC)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (4):
    densout = Dense(512, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(214, activation='linear')(densout)
densout = LeakyReLU(alpha=0.3)(densout)
surfout = SurRadLayer(
    inp_div=train_gen.input_transform.div,
    inp_sub=train_gen.input_transform.sub,
    norm_q=scale_dict['PHQ'],
    hyai=hyai, hybi=hybi
)([inpC, densout])
massout = MassConsLayer(
    inp_div=train_gen.input_transform.div,
    inp_sub=train_gen.input_transform.sub,
    norm_q=scale_dict['PHQ'],
    hyai=hyai, hybi=hybi
)([inpC, surfout])
enthout = EntConsLayer(
    inp_div=train_gen.input_transform.div,
    inp_sub=train_gen.input_transform.sub,
    norm_q=scale_dict['PHQ'],
    hyai=hyai, hybi=hybi
)([inpC, massout])
C_009 = tf.keras.models.Model(inpC, enthout)

## 3.2) Unconstrained Network (U)

In [9]:
# Unconstrained model with 5 dense layers
inpU = Input(shape=(304,))
densout = Dense(512, activation='linear')(inpU)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (4):
    densout = Dense(512, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(218, activation='linear')(densout)
out_layer = LeakyReLU(alpha=0.3)(densout)
U_009 = tf.keras.models.Model(inpU, out_layer)

## 3.3) Weakly-constrained Network (W)

In [10]:
# Weakly-constrained models with 5 dense layers
# alpha=0.01
inp001 = Input(shape=(304,))
densout = Dense(512, activation='linear')(inp001)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (4):
    densout = Dense(512, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(218, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
W001_009 = tf.keras.models.Model(inp001, out)
# alpha=0.5
inp05 = Input(shape=(304,))
densout = Dense(512, activation='linear')(inp05)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (4):
    densout = Dense(512, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(218, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
W05_009 = tf.keras.models.Model(inp05, out)
# alpha=0.99
inp099 = Input(shape=(304,))
densout = Dense(512, activation='linear')(inp099)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (4):
    densout = Dense(512, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(218, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
W099_009 = tf.keras.models.Model(inp099, out)

# 4) Compile and train the networks

## 4.1) Define losses
tgb - 4/10/2019 - Trying to solve the following error:  
  
You must feed a value for placeholder tensor 'input_2' with dtype float and shape [?,304]
	 [[{{node input_2}} = Placeholder[dtype=DT_FLOAT, shape=[?,304], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[{{node ConstantFoldingCtrl/loss/ent_cons_layer_loss/broadcast_weights/assert_broadcastable/AssertGuard/Switch_0/_310}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_972_C...d/Switch_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]  
  
when trying to train several networks in a row.
tgb - 4/10/2019 - First, understand how to access normalization  
  
tgb - 4/10/2019 - Found the problem: The weak loss function tracks a given input layer, and that has to be different for each network.

In [11]:
# # 1) Open the file containing the normalization of the targets
# ds = xr.open_dataset(TRAINDIR + PREFIX + 'norm.nc')
# # 2) Open the pickle files containing the pressure converters
# with open(os.path.join('/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/cbrain', 'hyai_hybi.pkl'), 'rb') as f:
#             hyai, hybi = pickle.load(f)
# print(ds)
# # 3) Define input transform and save it
# inp_transform = InputNormalizer(ds, in_vars, 'mean', 'maxrs')
# print(inp_transform)
# inp_div = inp_transform.div
# inp_sub = inp_transform.sub
# norm_q=scale_dict['PHQ']
# print('inp_div.shape=',inp_div.shape)
# print('inp_sub.shape=',inp_sub.shape)
# print('hyai.shape=',hyai.shape)
# print('hybi.shape=',hybi.shape)
# print('norm_q.shape=',norm_q.shape)

# ds.close()

In [12]:
W05_loss = WeakLoss(inp05, inp_div=train_gen.input_transform.div, inp_sub=train_gen.input_transform.sub,
                     norm_q=scale_dict['PHQ'], hyai=hyai, hybi=hybi, name='W05_loss')
W001_loss = WeakLoss(inp001, inp_div=train_gen.input_transform.div, inp_sub=train_gen.input_transform.sub,
                     norm_q=scale_dict['PHQ'], hyai=hyai, hybi=hybi, alpha_mass=5e-3, alpha_ent=5e-3, name = 'W001_loss')
W099_loss = WeakLoss(inp099, inp_div=train_gen.input_transform.div, inp_sub=train_gen.input_transform.sub,
                     norm_q=scale_dict['PHQ'], hyai=hyai, hybi=hybi, alpha_mass=0.99/2, alpha_ent=0.99/2, name = 'W099_loss')
masC_loss = WeakLoss(inpC, inp_div=train_gen.input_transform.div, inp_sub=train_gen.input_transform.sub,
                      norm_q=scale_dict['PHQ'], hyai=hyai, hybi=hybi, alpha_mass=1, alpha_ent=0, name='masC_loss')
entC_loss = WeakLoss(inpC, inp_div=train_gen.input_transform.div, inp_sub=train_gen.input_transform.sub,
                     norm_q=scale_dict['PHQ'], hyai=hyai, hybi=hybi, alpha_mass=0, alpha_ent=1, name='entC_loss')
masU_loss = WeakLoss(inpU, inp_div=train_gen.input_transform.div, inp_sub=train_gen.input_transform.sub,
                      norm_q=scale_dict['PHQ'], hyai=hyai, hybi=hybi, alpha_mass=1, alpha_ent=0, name='masU_loss')
entU_loss = WeakLoss(inpU, inp_div=train_gen.input_transform.div, inp_sub=train_gen.input_transform.sub,
                     norm_q=scale_dict['PHQ'], hyai=hyai, hybi=hybi, alpha_mass=0, alpha_ent=1, name='entU_loss')

## 4.2) Compile the models

### 4.2.1) Use mean square error as loss function

In [13]:
C_009.compile(tf.keras.optimizers.RMSprop(), loss=mse, metrics=[mse, masC_loss, entC_loss])

In [14]:
U_009.compile(tf.keras.optimizers.RMSprop(), loss=mse, metrics=[mse, masU_loss, entU_loss])

### 4.1.2) Use custom loss function

In [15]:
W001_009.compile(tf.keras.optimizers.RMSprop(), loss=W001_loss, metrics=[mass_loss, ent_loss, mse, W001_loss])

NameError: name 'mass_loss' is not defined

In [ ]:
W05_009.compile(tf.keras.optimizers.RMSprop(), loss=W05_loss, metrics=[mass_loss, ent_loss, mse, W05_loss])

In [ ]:
W099_009.compile(tf.keras.optimizers.RMSprop(), loss=W099_loss, metrics=[mass_loss, ent_loss, mse, W099_loss])

## 4.3) Train the models

### 4.3.1) Train compiled models

In [ ]:
# Nep = 10
# hC_009 = C_009.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen)
# hU_009 = U_009.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen)

### 4.3.2) Save models in .h5 format

In [ ]:
# %cd $TRAINDIR/HDF5_DATA
# !pwd
# C_009.save('C_009.h5')
# U_009.save('U_009.h5')

In [ ]:
# W001_rad_5dens.save('W001_rad2_5dens.h5')
# W05_rad_5dens.save('W05_rad2_5dens.h5')
# W099_rad_5dens.save('W099_rad2_5dens.h5')

### 4.3.3) Load models if already trained
tgb - 2/11/2019 - OPTIONAL: Load the models if already trained.

In [ ]:
%cd $TRAINDIR/HDF5_DATA
!ls
C_009.load_weights('C_009.h5')
U_009.load_weights('U_009.h5')

### 4.3.4) Loss and validation curves

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.subplot(111)

for index in range (2):
    if index==0: hdict = hC_009.history; colo = 'bo'; col = 'b'; lab = 'C';
    elif index==1: hdict = hU_009.history; colo = 'ro'; col = 'r'; lab = 'U';
    elif index==2: hdict = hW001_rad_5dens.history; colo = 'go'; col = 'g'; lab = 'W001';
    elif index==3: hdict = hW05_rad_5dens.history; colo = 'co'; col = 'c'; lab = 'W05';
    elif index==4: hdict = hW099_rad_5dens.history; colo = 'mo'; col = 'm'; lab = 'W099';
        
    
    train_loss_values = hdict['loss']
    valid_loss_values = hdict['val_loss']
    epochs = range(1, len(train_loss_values) + 1)

    ax.plot(epochs, train_loss_values, colo, label=lab+' Train')
    ax.plot(epochs, valid_loss_values, col, label=lab+' Valid')

#plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.ylim((0, 125))
# https://stackoverflow.com/questions/4700614/how-to-put-the-legend-out-of-the-plot
# for legend at the right place
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=5, fancybox=True, shadow=True); 
plt.show()

# 5) Compare R2 plot in present and future climate
tgb - 4/10/2019 - Use the new quickstart from Stephan at https://github.com/raspstephan/CBRAIN-CAM/blob/master/quickstart.ipynb

In [16]:
from cbrain.model_diagnostics.model_diagnostics import ModelDiagnostics

In [24]:
config_fn = '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local_PostProc.yml'
data_fn = '/local/Tom.Beucler/SPCAM_PHYS/8col009_01_valid.nc'
md = ModelDiagnostics(C_009,config_fn,data_fn)

OSError: [Errno -101] NetCDF: HDF error: b'/local/Tom.Beucler/SPCAM_PHYS/8col009_01_valid.nc'

In [22]:
import yaml
with open(config_fn, 'r') as f:
            config = yaml.load(f)

In [23]:
config

{'vars': ['QBP',
  'QCBP',
  'QIBP',
  'TBP',
  'VBP',
  'PS',
  'SOLIN',
  'SHFLX',
  'LHFLX',
  'PHQ',
  'PHCLDLIQ',
  'PHCLDICE',
  'TPHYSTND',
  'QRL',
  'QRS',
  'DTVKE',
  'FSNT',
  'FSNS',
  'FLNT',
  'FLNS',
  'PRECT',
  'PRECTEND',
  'PRECST',
  'PRECSTEN',
  'Qdt_adiabatic',
  'QCdt_adiabatic',
  'QIdt_adiabatic',
  'Tdt_adiabatic',
  'Vdt_adiabatic'],
 'in_dir': '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/',
 'in_fns': 'AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-*-0*-00000.nc',
 'out_dir': '/local/Tom.Beucler/SPCAM_PHYS/',
 'out_fn': '8col009_01_train.nc',
 'val_in_fns': 'AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-*-0*-00000.nc',
 'val_out_fn': '8col009_01_valid.nc',
 'norm_fn': '8col009_01_norm.nc'}